In [1]:
import os

import numpy as np
import itk
from itkwidgets import view

In [2]:
ROOT_DIR = '/home/stanislau/repository/builds/itk/ITK/Examples/Data/'

In [25]:
fixed_filename = os.path.join(ROOT_DIR, 'BrainT1SliceBorder20.png')
moving_filename = os.path.join(ROOT_DIR, 'BrainProtonDensitySliceR10X13Y17.png')

In [26]:
PixelType = itk.F
Dimension = 2

ImageType = itk.Image[PixelType, Dimension]

In [27]:
TTransformType = itk.TranslationTransform[itk.D, Dimension]

In [28]:
TOptimizerType = itk.RegularStepGradientDescentOptimizerv4[itk.D]
MetricType = itk.MattesMutualInformationImageToImageMetricv4[ImageType, ImageType]
TRegistrationType = itk.ImageRegistrationMethodv4[ImageType, ImageType]

In [29]:
trans_optimizer = TOptimizerType.New()
trans_metric = MetricType.New()
trans_registration = TRegistrationType.New()

In [30]:
trans_registration.SetOptimizer(trans_optimizer)
trans_registration.SetMetric(trans_metric)

In [31]:
moving_init_tx = TTransformType.New()

In [33]:
initial_parameters = itk.OptimizerParameters.D(2)
initial_parameters[0] = 3.0
initial_parameters[1] = 5.0

In [34]:
moving_init_tx.SetParameters(initial_parameters)

In [35]:
trans_registration.SetMovingInitialTransform(moving_init_tx)

In [36]:
CompositeTransformType = itk.CompositeTransform[itk.D, Dimension]
composite_transform = CompositeTransformType.New()
composite_transform.AddTransform(moving_init_tx)

In [37]:
fixed_image_reader = itk.ImageFileReader[ImageType].New()
moving_image_reader = itk.ImageFileReader[ImageType].New()

In [38]:
fixed_image_reader.SetFileName(fixed_filename)
moving_image_reader.SetFileName(moving_filename)

In [39]:
trans_registration.SetFixedImage(fixed_image_reader.GetOutput())
trans_registration.SetMovingImage(moving_image_reader.GetOutput())

In [40]:
trans_registration.SetNumberOfLevels(1)
trans_registration.SetShrinkFactorsPerLevel([3])
trans_registration.SetSmoothingSigmasPerLevel([2])

In [41]:
trans_metric.SetNumberOfHistogramBins(24)

In [42]:
trans_optimizer.SetNumberOfIterations(200)
trans_optimizer.SetRelaxationFactor(0.5)

In [43]:
trans_optimizer.SetLearningRate(16)
trans_optimizer.SetMinimumStepLength(1.5)

In [45]:
trans_registration.Update()

In [46]:
trans_metric.Initialize()